In [ ]:
import collections
import os

from immutables import Map
import pandas as pd
import plotly.express as px
import torch
from tqdm.auto import tqdm

from scrap_core import SUPPORTED_SCRAP_TYPES, vectorize_scrap_weights
from scrap_core.blendmodel.eob_config import BINNING
from scrap_core.blendmodel.eob_model_training import EOB_MODEL_SUPPORTED_CHEMS
from scrap_core.blendmodel.eob_datamodule import get_datamodule
from scrap_core.blendmodel.eob_model import OrdinalRegressionBlendModel
from scrap_core.blendmodel.eob_datamodule.eob_dataset import transform

In [ ]:
# Update parameters as needed

MODEL_VERSION = 16

MODEL_PATH = r"C:\Users\dec4288\Projects\UssAi\scrap_core\scrap_core\blendmodel\trained_models\eob_model_v{version}.pth"
CACHE_DIR = r"C:\Users\dec4288\Projects\UssAi\eob_model_training_cache"
OUTPUT_DIR = r"C:\Users\dec4288\data\eob_model\scrap_chem_content"  
           
subdir = os.path.join(OUTPUT_DIR, f"model_version_{MODEL_VERSION}")
if not os.path.exists(subdir):
    os.makedirs(subdir)

In [ ]:
pig_iron_weight = 140
scrap_weight = 40

In [ ]:
def create_input_vector(record: dict, chem: str) -> torch.Tensor:
    return torch.Tensor(
        [
            record[f"pig_iron_{chem.lower()}"],
            record["pig_iron_w"],
            record["pellets"],
            record["briquettes"],
            *vectorize_scrap_weights(SUPPORTED_SCRAP_TYPES, Map(record["scrap_map"])),
        ]
    )

## Export tables with model results for all supported chems

In [ ]:
data = []

for chem in EOB_MODEL_SUPPORTED_CHEMS:
    checkpoint = torch.load(MODEL_PATH.format(version=MODEL_VERSION))

    metadata = checkpoint[chem]

    model = OrdinalRegressionBlendModel(**metadata["model_config"])
    model.load_state_dict(metadata["state_dict"])
    model.eval()

    datamodule = get_datamodule(
    **{**metadata["dataset_params"], "oko_config": {}, "scada_config": {}, "path_cache": CACHE_DIR}
    )

    stats = {}

    for scrap in SUPPORTED_SCRAP_TYPES:
        heat = {
            f"pig_iron_{chem.lower()}": 0.002,
            "pig_iron_w": pig_iron_weight,
            "pellets": 0,
            "briquettes": 0,
            "scrap_map": {
                scrap: scrap_weight
            }
        }
        t = transform(create_input_vector(heat, chem), datamodule.summation_indexes).unsqueeze(dim=0)
        with torch.no_grad():
            stats[scrap] = model(t).squeeze().numpy().round(2)
            
    df = pd.DataFrame.from_dict(stats).T

    binning = metadata["dataset_params"]["binning"]
    
    df.columns = map(str, binning + ["inf"])
    
    df = df.sort_values(list(df.columns))
    
    low = torch.Tensor([0] + binning)
    
    df_low = pd.DataFrame.from_dict({st: (low * stats[st]).sum() for st in stats}, orient="index", columns=[chem])
    
    data.append(df_low)
        
    df.to_csv(os.path.join(subdir, f"scrap_{chem}_content.csv"))

## Display "expected" chem content per scrap

Values displayed on graphs in this section represent EXPECTED LOWER BOUNDS, i.e. expected content of given chem is at least the number displayed. Hence these results could be used to order scrap types based on chem content, but that is all.

In [ ]:
df_all = data[0]

for df in data[1:]:
    df_all = df_all.join(df)
    
df_all.head()

In [ ]:
fig = px.line(
    df_all.sort_index().reset_index().rename(columns={"index": "scrap"}),
    x="scrap",
    y=list(EOB_MODEL_SUPPORTED_CHEMS),
    title=f"expected lower bound per scrap"
)

fig.show()
fig.write_html(os.path.join(subdir, "expected_chem_content.html"))

## Display probality of reclasification for given scrap

In [ ]:
scrap = "TBC"

weights = list(range(scrap_weight + 1))
curves = collections.defaultdict(list)

for chem in EOB_MODEL_SUPPORTED_CHEMS:
    checkpoint = torch.load(MODEL_PATH.format(version=MODEL_VERSION))

    metadata = checkpoint[chem]

    model = OrdinalRegressionBlendModel(**metadata["model_config"])
    model.load_state_dict(metadata["state_dict"])
    model.eval()

    datamodule = get_datamodule(
    **{**metadata["dataset_params"], "oko_config": {}, "scada_config": {}, "path_cache": CACHE_DIR}
    )


    for weight in weights:
        heat = {
            f"pig_iron_{chem.lower()}": 0.002,
            "pig_iron_w": pig_iron_weight,
            "pellets": 0,
            "briquettes": 0,
            "scrap_map": {
                "HS": scrap_weight - weight,
                scrap: weight
            }
        }
        t = transform(create_input_vector(heat, chem), datamodule.summation_indexes).unsqueeze(dim=0)
        with torch.no_grad():
            curves[chem].append(model(t).squeeze()[-1])

In [ ]:
df_curves = pd.DataFrame.from_dict(curves)
    
fig = px.line(
    df_curves.reset_index().rename(columns={"index": scrap}),
    x=scrap,
    y=list(EOB_MODEL_SUPPORTED_CHEMS),
    title=f"probability of reclasification in synthetic heat"
)

fig.show()

## Export probablity of reclassification graphs for each scrap

In [ ]:
subdir_static = os.path.join(subdir, "static")
if not os.path.exists(subdir_static):
    os.mkdir(subdir_static)
    
    
subdir_interactive = os.path.join(subdir, "interactive")
if not os.path.exists(subdir_interactive):
    os.mkdir(subdir_interactive)
    

for scrap in tqdm(SUPPORTED_SCRAP_TYPES):
    weights = list(range(scrap_weight + 1))
    curves = collections.defaultdict(list)

    for chem in EOB_MODEL_SUPPORTED_CHEMS:
        checkpoint = torch.load(MODEL_PATH.format(version=MODEL_VERSION))

        metadata = checkpoint[chem]

        model = OrdinalRegressionBlendModel(**metadata["model_config"])
        model.load_state_dict(metadata["state_dict"])
        model.eval()

        datamodule = get_datamodule(
        **{**metadata["dataset_params"], "oko_config": {}, "scada_config": {}, "path_cache": CACHE_DIR}
        )


        for weight in weights:
            heat = {
                f"pig_iron_{chem.lower()}": 0.002,
                "pig_iron_w": pig_iron_weight,
                "pellets": 0,
                "briquettes": 0,
                "scrap_map": {
                    "HS": scrap_weight - weight,
                    scrap: weight
                }
            }
            t = transform(create_input_vector(heat, chem), datamodule.summation_indexes).unsqueeze(dim=0)
            with torch.no_grad():
                curves[chem].append(model(t).squeeze()[-1])
                
    df_curves = pd.DataFrame.from_dict(curves)

    fig = px.line(
        df_curves.reset_index().rename(columns={"index": scrap}),
        x=scrap,
        y=list(EOB_MODEL_SUPPORTED_CHEMS),
        title=f"probability of reclasification in synthetic heat"
    )

    fig.write_html(os.path.join(subdir_interactive, f"{scrap.replace(' ', '_')}_interactive.html"))
    fig.write_image(os.path.join(subdir_static, f"{scrap.replace(' ', '_')}_static.png"))